In [1]:
# install Pytorch

!pip3 install torch torchvision

    100% |████████████████████████████████| 591.8MB 28kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x6175c000 @  0x7f89735532a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 22.1MB/s 
    100% |████████████████████████████████| 2.0MB 5.0MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, sampler

import torchvision.datasets as dset 
import torchvision.transforms as transform 

In [12]:
# preprocess data

class CSampler(sampler.Sampler):
  
  def __init__(self, num_samp, start=0):
    self.num_samp  = num_samp
    self.start = start
    
    
  def __iter__(self):
    
    return iter(range(self.start, self.start + self.num_samp))
  
  
  def __Len__(self):
    
    return self.num_samp 



#data agumantation to the train data
train_transform = transform.Compose([
    transform.RandomHorizontalFlip(),
    transform.RandomRotation(15),
    transform.ToTensor()
])

# For test data

test_transform = transform.Compose([
    transform.ToTensor()
])


num_tarin = 49000
num_valid = 1000


CIFAR10_train = dset.CIFAR10(root = './data', train = True, download = True, 
                             transform=train_transform)


train_loder = DataLoader(CIFAR10_train, batch_size=128, sampler=CSampler(num_tarin, 0))

CIFAR10_val = dset.CIFAR10(root = './data', train = True, download = True, 
                             transform=test_transform)


val_loder = DataLoader(CIFAR10_val, batch_size=128, sampler=CSampler(num_valid, num_tarin))


CIFAR10_test = dset.CIFAR10(root = './data', train = False, download = True, 
                             transform=test_transform)


test_loder = DataLoader(CIFAR10_test, batch_size=128)



Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
torch.cuda.is_available()

True

In [0]:
# VGG16 Arch

class Flatten(nn.Module):
  def forward(self, x):
    N, C, H, W = x.size()
    return x.view(N, -1)
  
  
VGG16_model = nn.Sequential(# First Block 64
                            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(64),
                            nn.ReLU(inplace=True),
                            
                            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(64),
                            nn.ReLU(inplace=True),
                            
                            nn.MaxPool2d(kernel_size=2, stride=2),
                            
                            # Second block 128
                            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(128),
                            nn.ReLU(inplace=True),
                            
                            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(128),
                            nn.ReLU(inplace=True),
                            
                            nn.MaxPool2d(kernel_size=2, stride=2),
                            
                            #third block 256
                            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(256),
                            nn.ReLU(inplace=True),
                            
                            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(256),
                            nn.ReLU(inplace=True),
                            
                            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(256),
                            nn.ReLU(inplace=True),
                            
                            nn.MaxPool2d(kernel_size=2, stride=2),
                            
                            #forth block 512
                            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(512),
                            nn.ReLU(inplace=True),
                            
                            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(512),
                            nn.ReLU(inplace=True),
                            
                            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(512),
                            nn.ReLU(inplace=True),
                            
                            nn.MaxPool2d(kernel_size=2, stride=2),
                            
                            #fifth block 512
                            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(512),
                            nn.ReLU(inplace=True),
                            
                            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(512),
                            nn.ReLU(inplace=True),
                            
                            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(512),
                            nn.ReLU(inplace=True),
                            
                            nn.MaxPool2d(kernel_size=2, stride=2),
                            
                            # Flatte the output/ 512*1*1
                            Flatten(),
                            # Classefier
                            
                            nn.Linear(512, 10)
                           )

In [0]:
dtype = torch.cuda.FloatTensor

VGG16_model = VGG16_model.type(dtype)

lss_fun = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(VGG16_model.parameters(), lr=1e-3, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=.1) 

In [0]:
# function to train the model for 1 epoch on the data set
def train(model, loder, lss_fn, optmizer, epoch):
  print('epoch number %d' % (epoch))
  model.train()
  epoch_loss = 0.0
  epoch_acc = 0.0
  num_itr = 0
  for idx, (x, y) in enumerate(loder):
    # move to gpu
    x = x.type(dtype)
    y = y.to('cuda')
    #compute the ouput of the model
    score = model(x)
    # compute loss
    loss = lss_fn(score, y)
    #compute the gradiant and update the weights
    optmizer.zero_grad()
    loss.backward()
    optmizer.step()
    # accomulate the loss and acc
    epoch_loss += loss.item()
    _, prediction = score.max(1)
    epoch_acc += y.eq(prediction).sum().item() / y.size(0)
    num_itr = idx+1
  
  epoch_loss /= num_itr
  epoch_acc /= num_itr
  
  return epoch_loss, epoch_acc

In [0]:
def compte_acc(model, loder, lss_fn):
  
  if loder.dataset.train:
    print("accurcy of Validation set")
  else:
    print("accurcy of test set")
  
  loss = 0.0
  acc = 0.0
  num_itr = 0
  for idx, (x, y) in enumerate(loder):
    x = x.type(dtype)
    y = y.to('cuda')
    score = model(x)
    loss += lss_fn(score, y).item()
    _, prid = score.max(1)
    acc += (prid == y).sum().item() / y.size(0)
    num_itr = idx+1
    
  acc /= num_itr
  loss /= num_itr
  
  return loss, acc

In [0]:
# loop of epochs 

for i in range(100):
  scheduler.step()
  %time tr_loss, tr_acc = train(VGG16_model, train_loder, lss_fun, optimizer, i)
  print(f'traning loss {tr_loss} and traning acc {tr_acc}')
  
  val_loss, val_acc = compte_acc(VGG16_model, val_loder, lss_fun)
  print(f'val loss {val_loss} and val acc {val_acc}')

epoch number 0
CPU times: user 2min 59s, sys: 22.4 s, total: 3min 21s
Wall time: 3min 21s
traning loss 0.5136792517362002 and traning acc 0.8226943788913437
accurcy of Validation set
val loss 0.511946901679039 and val acc 0.8318810096153846
epoch number 1
CPU times: user 2min 59s, sys: 22.3 s, total: 3min 21s
Wall time: 3min 21s
traning loss 0.5115465984798909 and traning acc 0.8238209856396866
accurcy of Validation set
val loss 0.5086415968835354 and val acc 0.8333834134615384
epoch number 2
CPU times: user 2min 59s, sys: 22.2 s, total: 3min 21s
Wall time: 3min 21s
traning loss 0.508135883366159 and traning acc 0.824392134464752
accurcy of Validation set
val loss 0.5063333623111248 and val acc 0.8345853365384616
epoch number 3
CPU times: user 2min 59s, sys: 22.1 s, total: 3min 21s
Wall time: 3min 21s
traning loss 0.5042048346436054 and traning acc 0.8249805432817835
accurcy of Validation set
val loss 0.5048054233193398 and val acc 0.8343599759615384
epoch number 4
CPU times: user 2min